In [7]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')
save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_checkpoints')
save_dir.mkdir(parents=True, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install ultralytics

In [9]:
from ultralytics import YOLO

import matplotlib.pyplot as plt

In [10]:
pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0


In [11]:
from roboflow import Roboflow
from ultralytics import YOLO
import os

rf = Roboflow(api_key="HA7uKn41BYAjKQOtGKJz")
project = rf.workspace("capstone-ncwsi").project("test-5dvot")
version = project.version("17")

# download dataset
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.66, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to test-17 in yolov8:: 100%|██████████| 14386/14386 [00:02<00:00, 5261.03it/s]


In [12]:
import yaml

yaml_path = os.path.join(dataset.location, "data.yaml")
print(dataset.location)

with open(yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)
print(data_config)

print("YAML Path:", yaml_path)

/content/test-17
{'names': ['Beetle-fire Tree', 'Dead Tree', 'debris'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'test-5dvot', 'url': 'https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/17', 'version': 17, 'workspace': 'capstone-ncwsi'}, 'test': '../test/images', 'train': 'test-17/train/images', 'val': 'test-17/valid/images'}
YAML Path: /content/test-17/data.yaml


In [13]:
yaml_path = '/content/test-17/data.yaml'
with open(yaml_path, 'r') as file:
    yaml_content = yaml.safe_load(file)

yaml_content['train'] = 'train/images'
yaml_content['val'] = 'valid/images'
yaml_content['test'] = 'test/images'

with open(yaml_path, 'w') as file:
    yaml.dump(yaml_content, file, default_flow_style=False)

print("Updated YAML file contents:")
with open(yaml_path, 'r') as file:
    print(file.read())

Updated YAML file contents:
names:
- Beetle-fire Tree
- Dead Tree
- debris
nc: 3
roboflow:
  license: CC BY 4.0
  project: test-5dvot
  url: https://universe.roboflow.com/capstone-ncwsi/test-5dvot/dataset/17
  version: 17
  workspace: capstone-ncwsi
test: test/images
train: train/images
val: valid/images



In [14]:
model = YOLO(model='yolov8m.pt')

current_dir = '/content/test-17'
yaml_path = os.path.join(current_dir, 'data.yaml')

100%|██████████| 49.7M/49.7M [00:00<00:00, 312MB/s]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from ultralytics import YOLO



# Define multi-IoU threshold evaluation function
def evaluate_model_multi_iou(model, data_path, iou_thresholds):
    results = []
    for iou in iou_thresholds:
        result = model.val(data=data_path, iou=iou)
        results.append(result)

    class_names = results[0].names
    class_maps = {name: [] for name in class_names.values()}

    for result in results:
        for i, name in class_names.items():
            class_maps[name].append(result.box.maps[i])

    average_map = np.mean([result.box.map for result in results])

    print(f"Average mAP across IOU thresholds: {average_map:.4f}")

    for name, maps in class_maps.items():
        print(f"{name} average mAP: {np.mean(maps):.4f}")

    plt.figure(figsize=(10, 6))
    for name, maps in class_maps.items():
        plt.plot(iou_thresholds, maps, label=name, marker='o')

    plt.plot(iou_thresholds, [result.box.map for result in results], label='Overall', marker='o', linestyle='--', linewidth=2)

    plt.title('mAP vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('mAP')
    plt.legend()
    plt.grid(True)
    plt.savefig('mAP_vs_IOU.png')
    plt.close()

    precisions = np.array([result.box.p for result in results])
    recalls = np.array([result.box.r for result in results])

    plt.figure(figsize=(10, 6))
    for i, name in enumerate(class_names.values()):
        plt.plot(iou_thresholds, precisions[:, i], label=f'{name} Precision', marker='o')
        plt.plot(iou_thresholds, recalls[:, i], label=f'{name} Recall', marker='o', linestyle='--')
    plt.title('Precision and Recall vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.savefig('Precision_Recall_vs_IOU.png')
    plt.close()

    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-16)  # 添加小量以避免除零

    plt.figure(figsize=(10, 6))
    for i, name in enumerate(class_names.values()):
        plt.plot(iou_thresholds, f1_scores[:, i], label=f'{name} F1 Score', marker='o')
    plt.title('F1 Score vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True)
    plt.savefig('F1_Score_vs_IOU.png')
    plt.close()

    return average_map, class_maps

# callback function to save results
def on_train_epoch_end(trainer):
    epoch = trainer.epoch
    if epoch % trainer.args.save_period == 0:
        best_fitness = trainer.best_fitness if trainer.best_fitness is not None else 0.0
        extra_info = {
            'epoch': epoch,
            'best_fitness': float(best_fitness),
            'learning_rate': float(trainer.optimizer.param_groups[0]['lr']),
        }
        save_dir = Path(trainer.args.project) / trainer.args.name
        info_path = save_dir / f'extra_info_epoch_{epoch}.json'
        latest_info_path = save_dir / 'extra_info_latest.json'
        with open(info_path, 'w') as f:
            json.dump(extra_info, f)
        with open(latest_info_path, 'w') as f:
            json.dump(extra_info, f)

# train and evaluate
def train_and_evaluate():
    yaml_path = '/content/test-17/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_checkpoints2')
    best_model_path = save_dir / 'run_from_scratch/weights/best.pt'

    
    if best_model_path.exists():
        print(f"Loading best model from {best_model_path}")
        model = CustomYOLO(str(best_model_path))
    else:
        print("Starting training from scratch")
        model = CustomYOLO('yolov8m.pt')

    
    model.add_callback('on_train_epoch_end', on_train_epoch_end)

# def train_and_evaluate():
#     yaml_path = '/content/test-17/data.yaml'
#     save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_checkpoints2')

#     model = YOLO('yolov8m.pt')
#     print("Starting training from scratch")

    # checkpoint = save_dir / 'run2' / 'weights' / 'epoch140.pt'
    # if checkpoint.exists():
    #     print(f"Resuming from checkpoint: {checkpoint}")
    #     model = YOLO(str(checkpoint))
    #     completed_epochs = 140
    # else:
    #     print(f"Checkpoint {checkpoint} not found. Please check the file path.")
    #     return

    # total_epochs = 200
    # remaining_epochs = total_epochs - completed_epochs

    # print(f"Completed epochs: {completed_epochs}")
    # print(f"Remaining epochs: {remaining_epochs}")

    
    # model.add_callback('on_train_epoch_end', on_train_epoch_end)

    # results = model.train(
    #     data=yaml_path,
    #     epochs=200,
    #     imgsz=640,
    #     batch=32,
    #     lr0=0.001,
    #     lrf=0.01,
    #     momentum=0.937,
    #     weight_decay=0.001,
    #     warmup_epochs=5,
    #     warmup_bias_lr=0.1,
    #     warmup_momentum=0.8,
    #     box=10.0,
    #     cls=1.0,
    #     dfl=1.5,
    #     patience=100,
    #     optimizer="AdamW",
    #     cos_lr=True,
    #     mixup=0.5,
    #     copy_paste=0.3,
    #     degrees=15.0,
    #     hsv_h=0.015,
    #     hsv_s=0.7,
    #     hsv_v=0.4,
    #     translate=0.2,
    #     scale=0.6,
    #     shear=3.0,
    #     perspective=0.0005,
    #     flipud=0.5,
    #     fliplr=0.5,
    #     mosaic=1.0,
    #     conf=0.001,
    #     iou=0.5,
    #     verbose=True,
    #     save_period=20,
    #     project=str(save_dir),
    #     name='run_from_scratch',
    #     exist_ok=True,
    #     multi_scale=True,
    #     label_smoothing=0.1,
    #     device='0'
    # )

  results = model.train(
        data=yaml_path,
        epochs=200,
        imgsz=640,
        batch=32,
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.001,
        warmup_epochs=5,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        box=10.0,
        cls=1.0,
        dfl=1.5,
        patience=100,
        optimizer="AdamW",
        cos_lr=True,
        mixup=0.5,
        copy_paste=0.3,
        degrees=15.0,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        translate=0.2,
        scale=0.6,
        shear=3.0,
        perspective=0.0005,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        conf=0.001,
        iou=0.5,
        verbose=True,
        save_period=20,
        project=str(save_dir),
        name='run_with_focal_loss',
        exist_ok=True,
        multi_scale=True,
        label_smoothing=0.1,
        device='0'
    )


    # multi-IOU evaluate
    data_path = '/content/test-17'
    iou_thresholds = np.arange(0.2, 1.0, 0.05)
    average_map, class_maps = evaluate_model_multi_iou(model, data_path, iou_thresholds)

    for name, maps in class_maps.items():
        print(f"\n{name}:")
        for iou, map_value in zip(iou_thresholds, maps):
            print(f"  IOU {iou:.2f}: mAP {map_value:.4f}")


if __name__ == "__main__":
    train_and_evaluate()


Starting training from scratch
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/test-17/data.yaml, epochs=200, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=20, cache=False, device=0, workers=8, project=/content/drive/MyDrive/YOLOv8_nadir_checkpoints2, name=run_from_scratch, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

100%|██████████| 755k/755k [00:00<00:00, 115MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 343MB/s]


AMP: checks passed ✅


...

                   all        607       1447      0.395      0.346      0.271      0.104
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 59, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



159 epochs completed in 2.376 hours.
Optimizer stripped from /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch/weights/best.pt...
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:05<00:00,  1.98it/s]


                   all        607       1447       0.43      0.391      0.328      0.132
      Beetle-fire Tree         63         79      0.515      0.481       0.41      0.188
             Dead Tree        297        550      0.479      0.447      0.403      0.154
                debris        265        818      0.297      0.243      0.172     0.0539
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


FileNotFoundError: Dataset '/content/test-17' for task=detect not found ❌

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from ultralytics import YOLO

# Define multi-IoU threshold evaluation function
def evaluate_model_multi_iou(model_path, data_path, iou_thresholds):
    model = YOLO(model_path)
    results = []
    for iou in iou_thresholds:
        result = model.val(data=data_path, iou=iou)
        results.append(result)

    class_names = results[0].names
    class_maps = {name: [] for name in class_names.values()}

    for result in results:
        for i, name in class_names.items():
            class_maps[name].append(result.box.maps[i])

    average_map = np.mean([result.box.map for result in results])

    print(f"Average mAP across IOU thresholds: {average_map:.4f}")

    for name, maps in class_maps.items():
        print(f"{name} average mAP: {np.mean(maps):.4f}")

    plt.figure(figsize=(10, 6))
    for name, maps in class_maps.items():
        plt.plot(iou_thresholds, maps, label=name, marker='o')

    plt.plot(iou_thresholds, [result.box.map for result in results], label='Overall', marker='o', linestyle='--', linewidth=2)

    plt.title('mAP vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('mAP')
    plt.legend()
    plt.grid(True)
    plt.savefig('mAP_vs_IOU.png')
    plt.close()

    precisions = np.array([result.box.p for result in results])
    recalls = np.array([result.box.r for result in results])

    plt.figure(figsize=(10, 6))
    for i, name in enumerate(class_names.values()):
        plt.plot(iou_thresholds, precisions[:, i], label=f'{name} Precision', marker='o')
        plt.plot(iou_thresholds, recalls[:, i], label=f'{name} Recall', marker='o', linestyle='--')
    plt.title('Precision and Recall vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.savefig('Precision_Recall_vs_IOU.png')
    plt.close()

    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-16)  # 添加小量以避免除零

    plt.figure(figsize=(10, 6))
    for i, name in enumerate(class_names.values()):
        plt.plot(iou_thresholds, f1_scores[:, i], label=f'{name} F1 Score', marker='o')
    plt.title('F1 Score vs IOU Threshold')
    plt.xlabel('IOU Threshold')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True)
    plt.savefig('F1_Score_vs_IOU.png')
    plt.close()

    return average_map, class_maps

# callback function to save results
def on_train_epoch_end(trainer):
    epoch = trainer.epoch
    if epoch % trainer.args.save_period == 0:
        best_fitness = trainer.best_fitness if trainer.best_fitness is not None else 0.0
        extra_info = {
            'epoch': epoch,
            'best_fitness': float(best_fitness),
            'learning_rate': float(trainer.optimizer.param_groups[0]['lr']),
        }
        save_dir = Path(trainer.args.project) / trainer.args.name
        info_path = save_dir / f'extra_info_epoch_{epoch}.json'
        latest_info_path = save_dir / 'extra_info_latest.json'
        with open(info_path, 'w') as f:
            json.dump(extra_info, f)
        with open(latest_info_path, 'w') as f:
            json.dump(extra_info, f)

# train and evaluate
def train_and_evaluate():
    yaml_path = '/content/test-17/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_checkpoints2')
    best_model_path = save_dir / 'run_from_scratch/weights/best.pt'

    # load best model
    if best_model_path.exists():
        print(f"Loading best model from {best_model_path}")
        model = YOLO(str(best_model_path))
    else:
        print("Best model not found. Starting training from scratch.")
        model = YOLO('yolov8m.pt')


        model.add_callback('on_train_epoch_end', on_train_epoch_end)

        # train
        results = model.train(
            data=yaml_path,
            epochs=200,
            imgsz=640,
            batch=32,
            lr0=0.001,
            lrf=0.01,
            momentum=0.937,
            weight_decay=0.001,
            warmup_epochs=5,
            warmup_bias_lr=0.1,
            warmup_momentum=0.8,
            box=10.0,
            cls=1.0,
            dfl=1.5,
            patience=100,
            optimizer="AdamW",
            cos_lr=True,
            mixup=0.5,
            copy_paste=0.3,
            degrees=15.0,
            hsv_h=0.015,
            hsv_s=0.7,
            hsv_v=0.4,
            translate=0.2,
            scale=0.6,
            shear=3.0,
            perspective=0.0005,
            flipud=0.5,
            fliplr=0.5,
            mosaic=1.0,
            conf=0.001,
            iou=0.5,
            verbose=True,
            save_period=20,
            project=str(save_dir),
            name='run_from_scratch',
            exist_ok=True,
            multi_scale=True,
            label_smoothing=0.1,
            device='0'
        )

    
    iou_thresholds = np.arange(0.2, 1.0, 0.05)
    average_map, class_maps = evaluate_model_multi_iou(str(best_model_path), yaml_path, iou_thresholds)

    
    for name, maps in class_maps.items():
        print(f"\n{name}:")
        for iou, map_value in zip(iou_thresholds, maps):
            print(f"  IOU {iou:.2f}: mAP {map_value:.4f}")

if __name__ == "__main__":
    train_and_evaluate()

Loading best model from /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch/weights/best.pt
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:05<00:00,  6.49it/s]


                   all        607       1447      0.441      0.381      0.306      0.123
      Beetle-fire Tree         63         79      0.526      0.456      0.386      0.176
             Dead Tree        297        550      0.495      0.442       0.38      0.144
                debris        265        818      0.303      0.246      0.153     0.0479
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/val17
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:05<00:00,  6.79it/s]


                   all        607       1447      0.439      0.381      0.311      0.124
      Beetle-fire Tree         63         79      0.518      0.456      0.389      0.177
             Dead Tree        297        550      0.495      0.442      0.388      0.146
                debris        265        818      0.303      0.246      0.157     0.0489
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val18
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


...

                   all        607       1447       0.43      0.392      0.328      0.131
      Beetle-fire Tree         63         79      0.513      0.481       0.41      0.187
             Dead Tree        297        550      0.479      0.449      0.403      0.154
                debris        265        818      0.299      0.246      0.172      0.054
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/val23
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:05<00:00,  6.96it/s]


                   all        607       1447       0.43      0.388      0.325      0.131
      Beetle-fire Tree         63         79      0.518      0.481      0.406      0.184
             Dead Tree        297        550      0.471      0.445      0.396      0.155
                debris        265        818      0.301      0.237      0.172     0.0546
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/val24
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  11%|█         | 4/38 [00:02<00:21,  1.56it/s]


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

class CustomYOLO(YOLO):
    def __init__(self, model='yolov8n.pt', task=None):
        super().__init__(model, task)
        self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)

    def _load(self, weights='yolov8n.pt', task='detect'):
        super()._load(weights, task)
        # focal loss
        if hasattr(self.model, 'model') and hasattr(self.model.model, 'loss'):
            original_loss_func = self.model.model.loss
            def custom_loss(*args, **kwargs):
                loss, loss_items = original_loss_func(*args, **kwargs)
                
                pred_scores, target_scores = args[0][1], args[1]['cls']
                focal_loss = self.focal_loss(pred_scores, target_scores)
                loss_items[1] = focal_loss  
                loss = loss_items.sum()
                return loss, loss_items
            self.model.model.loss = custom_loss

def train_and_evaluate():
    yaml_path = '/content/test-17/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_checkpoints2')
    best_model_path = save_dir / 'run_from_scratch/weights/best.pt'

    
    if best_model_path.exists():
        print(f"Loading best model from {best_model_path}")
        model = CustomYOLO(str(best_model_path))
    else:
        print("Starting training from scratch")
        model = CustomYOLO('yolov8m.pt')

    
    model.add_callback('on_train_epoch_end', on_train_epoch_end)

    
    results = model.train(
        data=yaml_path,
        epochs=200,
        imgsz=640,
        batch=32,
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.001,
        warmup_epochs=5,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        box=10.0,
        cls=1.0,
        dfl=1.5,
        patience=100,
        optimizer="AdamW",
        cos_lr=True,
        mixup=0.5,
        copy_paste=0.3,
        degrees=15.0,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        translate=0.2,
        scale=0.6,
        shear=3.0,
        perspective=0.0005,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        conf=0.001,
        iou=0.5,
        verbose=True,
        save_period=20,
        project=str(save_dir),
        name='run_with_focal_loss',
        exist_ok=True,
        multi_scale=True,
        label_smoothing=0.1,
        device='0'
    )
    
    iou_thresholds = np.arange(0.2, 1.0, 0.05)
    average_map, class_maps = evaluate_model_multi_iou(str(best_model_path), yaml_path, iou_thresholds)

    
    for name, maps in class_maps.items():
        print(f"\n{name}:")
        for iou, map_value in zip(iou_thresholds, maps):
            print(f"  IOU {iou:.2f}: mAP {map_value:.4f}")

if __name__ == "__main__":
    train_and_evaluate()


Loading best model from /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch/weights/best.pt
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch/weights/best.pt, data=/content/test-17/data.yaml, epochs=200, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=20, cache=False, device=0, workers=8, project=/content/drive/MyDrive/YOLOv8_nadir_checkpoints2, name=run_with_focal_loss, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stri

train: Scanning /content/test-17/train/labels.cache... 6282 images, 1515 backgrounds, 0 corrupt: 100%|██████████| 6282/6282 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_with_focal_loss/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.001), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_with_focal_loss
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        607       1447      0.327      0.314      0.241      0.107
      Beetle-fire Tree         63         79       0.38      0.405      0.319      0.154
             Dead Tree        297        550      0.324      0.375      0.282      0.124
                debris        265        818      0.278      0.163      0.121     0.0427
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val39
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:07<00:00,  5.34it/s]


                   all        607       1447      0.261      0.292      0.198     0.0905
      Beetle-fire Tree         63         79      0.285       0.38      0.266      0.133
             Dead Tree        297        550      0.261      0.356      0.233      0.104
                debris        265        818      0.236      0.141     0.0961     0.0346
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs/detect/val40
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/test-17/valid/labels.cache... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:10<00:00,  3.59it/s]


                   all        607       1447       0.18      0.262      0.138     0.0652
      Beetle-fire Tree         63         79      0.182      0.354      0.194        0.1
             Dead Tree        297        550      0.188      0.331      0.156     0.0715
                debris        265        818      0.171      0.101     0.0641     0.0238
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to runs/detect/val41
Average mAP across IOU thresholds: 0.1198
Beetle-fire Tree average mAP: 0.1705
Dead Tree average mAP: 0.1402
debris average mAP: 0.0486

Beetle-fire Tree:
  IOU 0.20: mAP 0.1762
  IOU 0.25: mAP 0.1770
  IOU 0.30: mAP 0.1795
  IOU 0.35: mAP 0.1813
  IOU 0.40: mAP 0.1840
  IOU 0.45: mAP 0.1851
  IOU 0.50: mAP 0.1866
  IOU 0.55: mAP 0.1837
  IOU 0.60: mAP 0.1835
  IOU 0.65: mAP 0.1849
  IOU 0.70: mAP 0.1820
  IOU 0.75: mAP 0.1734
  IOU 0.80: mAP 0.1636
  IOU 0.85: mAP 0.1545
  IOU 0.90: mAP 0.1327
  IOU 0.95: mAP 0.1003

Dea

In [4]:
from google.colab import drive
import os

drive.mount('/content/drive')


base_path = '/content/drive/MyDrive/YOLOv8_nadir_checkpoints2'
print(f"Contents of {base_path}:")
print(os.listdir(base_path))


run_from_scratch_path = os.path.join(base_path, 'run_from_scratch')
if os.path.exists(run_from_scratch_path):
    print(f"\nContents of {run_from_scratch_path}:")
    print(os.listdir(run_from_scratch_path))

    
    weights_path = os.path.join(run_from_scratch_path, 'weights')
    if os.path.exists(weights_path):
        print(f"\nContents of {weights_path}:")
        print(os.listdir(weights_path))

Mounted at /content/drive
Contents of /content/drive/MyDrive/YOLOv8_nadir_checkpoints2:
['run_from_scratch', 'run_with_focal_loss']

Contents of /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch:
['weights', 'args.yaml', 'labels_correlogram.jpg', 'labels.jpg', 'train_batch0.jpg', 'train_batch1.jpg', 'train_batch2.jpg', 'extra_info_epoch_0.json', 'extra_info_epoch_20.json', 'extra_info_epoch_40.json', 'extra_info_epoch_60.json', 'extra_info_epoch_80.json', 'extra_info_epoch_100.json', 'extra_info_epoch_120.json', 'extra_info_epoch_140.json', 'extra_info_latest.json', 'val_batch0_pred.jpg', 'val_batch0_labels.jpg', 'val_batch1_labels.jpg', 'val_batch1_pred.jpg', 'results.csv', 'val_batch2_labels.jpg', 'val_batch2_pred.jpg', 'PR_curve.png', 'F1_curve.png', 'P_curve.png', 'R_curve.png', 'confusion_matrix_normalized.png', 'confusion_matrix.png', 'results.png', 'events.out.tfevents.1722032975.9d9cfa8baecf.6640.0']

Contents of /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/

In [16]:

from pathlib import Path

def evaluate_model():
    yaml_path = '/content/test-17/data.yaml'
    save_dir = Path('/content/drive/MyDrive/YOLOv8_nadir_checkpoints2')

    # load best.pt
    best_model_path = save_dir / 'run_from_scratch/weights/best.pt'
    if best_model_path.exists():
        print(f"Loading model from: {best_model_path}")
        model = YOLO(str(best_model_path))
    else:
        print(f"Error: Model file not found at {best_model_path}")
        return

    
    results = model.val(data=yaml_path)

   
    print("\nValidation Results:")
    print(f"mAP50: {results.box.map50:.4f}")
    print(f"mAP50-95: {results.box.map:.4f}")

    
    for i, class_name in enumerate(results.names.values()):
        print(f"\n{class_name}:")
        print(f"  Precision: {results.box.p[i]:.4f}")
        print(f"  Recall: {results.box.r[i]:.4f}")
        print(f"  mAP50: {results.box.maps[i]:.4f}")

if __name__ == "__main__":
    evaluate_model()

Loading model from: /content/drive/MyDrive/YOLOv8_nadir_checkpoints2/run_from_scratch/weights/best.pt
Ultralytics YOLOv8.2.66 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 63.9MB/s]
val: Scanning /content/test-17/valid/labels... 607 images, 157 backgrounds, 0 corrupt: 100%|██████████| 607/607 [00:00<00:00, 1347.27it/s]

val: New cache created: /content/test-17/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:07<00:00,  5.25it/s]


                   all        607       1447      0.427      0.362      0.307      0.129
      Beetle-fire Tree         63         79       0.52      0.468      0.394      0.182
             Dead Tree        297        550      0.438      0.416      0.365      0.151
                debris        265        818      0.323      0.201      0.162      0.054
Speed: 0.4ms preprocess, 2.9ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to runs/detect/val3

Validation Results:
mAP50: 0.3069
mAP50-95: 0.1289

Beetle-fire Tree:
  Precision: 0.5202
  Recall: 0.4684
  mAP50: 0.1820

Dead Tree:
  Precision: 0.4384
  Recall: 0.4164
  mAP50: 0.1506

debris:
  Precision: 0.3225
  Recall: 0.2008
  mAP50: 0.0540
